# Imports

In [42]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Constants

In [43]:
torch.manual_seed(1337)
training_style = "scale"
if training_style == "scale":
    block_size = 32
    batch_size = 512
    max_iters = 5000
    eval_interval = 500
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 384
    n_layer = 6 # Number of block layers
    n_head = 6
    dropout = 0.2
else:
    block_size = 32
    batch_size = 256
    split_pct = 0.9
    max_iters = 500
    eval_interval = 100
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 64
    n_layer = 2 # Number of block layers
    n_head = 2
    dropout = 0.2

In [44]:
# Set GPU device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(device)

cuda


# Dataloader

In [45]:
chars = ['\\', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '.', '=', 'e']
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 16


In [46]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] if c in chars else print (s) for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [47]:
def get_batch(batch_size):
    numbers = torch.randint(0, 1000000, (batch_size+4, 2))
    data = ""
    ix = []
    for idx in range(numbers.size(0)):
        eqn = "{}+{}={}".format(numbers[idx, 0], numbers[idx, 1], numbers[idx, 0]+numbers[idx, 1])
        if idx > 2 and idx < (numbers.size()[0] - 1):
            ix.append(torch.randint(len(data) + eqn.index("=") + 2, len(data) + len(eqn) + 1, (1,)).item())
        data = data + "\\" + eqn
    data = torch.tensor(encode(data))
    x = torch.stack([data[i-block_size:i] for i in ix])
    # y here is the target - the next token for any given point in x
    y = torch.stack([data[i-block_size+1:i+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [48]:
x, y = get_batch(batch_size)
x.size(), y.size()

(torch.Size([512, 32]), torch.Size([512, 32]))

In [49]:
for idx in range(0, len(x), 50):
    print(decode(x[idx].tolist()), decode(y[idx].tolist()))

13753=1058048\751015+906010=1657 3753=1058048\751015+906010=16570
35+972989=1563524\753748+689526= 5+972989=1563524\753748+689526=1
049=1087731\994298+696814=169111 49=1087731\994298+696814=1691112
7+791227=1135974\94533+784841=87 +791227=1135974\94533+784841=879
07+708025=873432\468402+179764=6 7+708025=873432\468402+179764=64
+196196=654701\650051+753387=140 196196=654701\650051+753387=1403
7692=1838457\383909+304666=68857 692=1838457\383909+304666=688575
01+521210=1117811\838628+304607= 1+521210=1117811\838628+304607=1
03+976020=1671623\644434+787754= 3+976020=1671623\644434+787754=1
4747=253879\774788+604491=137927 747=253879\774788+604491=1379279
0+132736=360206\142695+738086=88 +132736=360206\142695+738086=880


# Model

In [50]:
# Obtains average loss over multiple batches
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, T = get_batch(batch_size)
            logits, loss = m(X, T)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [51]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # (C, 3C)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        self.dropout1 = nn.Dropout(dropout)
        self.proj = nn.Linear(n_embd, n_embd) # (C, C)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # x has shape (B,T,C) - because of initial embeddings
        B, T, C = x.shape

        q, k, v = self.c_attn(x).split(n_embd, dim=2) # (B, T, C) @ (C, 3C) --> (B, T, 3C) splits -> (B, T, C)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)

        # affinities
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, nh, T, hs) @ (B, nh, hs, T) ---> (B,nh,T,T)
        wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf')) # (B, nh, T, T)
        wei = F.softmax(wei, dim=-1) # (B, nh, T, T)
        wei = self.dropout1(wei) # (B, nh, T, T)
        out = wei @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B,T,C) # (B, T, nh, hs) -> (B, T, C)
        out = self.dropout2(self.proj(out)) # (B, T, C) @ (C, C) --> (B, T, C)
        return out

In [52]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [53]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [54]:
class GptModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tkn_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tkn_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)#, ignore_index=encode("\\")[0])
            
        return logits, loss

    def generate(self, idx):
        # idx is (B, T) array of indices in the current context
        for _ in range(10):
            # crop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # )B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [55]:
m = GptModel()
m.to(device)

GptModel(
  (token_embedding_table): Embedding(16, 384)
  (position_embedding_table): Embedding(32, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (dropout1): Dropout(p=0.2, 

In [57]:
# Number of parameters in model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

TypeError: 'str' object is not callable

# Training

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch(batch_size)
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.8964, val loss 2.8962
step 500: train loss 2.0354, val loss 2.0342
step 1000: train loss 1.9974, val loss 1.9978
step 1500: train loss 1.9857, val loss 1.9853
step 2000: train loss 1.9047, val loss 1.9055
step 2500: train loss 1.8618, val loss 1.8615
step 3000: train loss 1.8434, val loss 1.8435
step 3500: train loss 1.8369, val loss 1.8372
step 4000: train loss 1.8317, val loss 1.8324
step 4500: train loss 1.8306, val loss 1.8310
step 4999: train loss 1.8291, val loss 1.8303


# Inference

In [ ]:
def generate(idx):
    # idx is (B, T) array of indices in the current context
    for _ in range(10):
        # crop idx to last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get predictions
        logits, loss = m(idx_cond)
        # focus only on last time step
        logits = logits[:, -1, :]
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # )B, C)
        # sample from distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

In [ ]:
val1 = 12345
val2 = 678910
sum = "{}+{}=".format(val1, val2)
context = torch.tensor(encode(sum)).unsqueeze(0).to(device)
print("Result: {}".format(val1 + val2))
print(decode(generate(context)[0].tolist()))

Result: 691255
12345+678910=7708052721


# Save the model

In [ ]:
#import time
#torch.save(m.state_dict(), './models/addition_calculator_{}.pt'.format(time.strftime("%Y-%m-%d-%H:%M")))